# Fitting All Wavelengths Using PandExo and Juliet 

### Import Libraries

In [1]:
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d
from scipy.interpolate import interp1d
import numpy as np
import batman
import juliet
import pickle

In [2]:
data = pickle.load(open('data/ETC-calculationc0e1784d-d070-49bd-9536-8c5183c58fa4e.p','rb'))

In [3]:
var_out, var_in = data['RawData']['var_out'], data['RawData']['var_in']
S_out, S_in = data['RawData']['electrons_out'], data['RawData']['electrons_in']
N_out, N_in = data['timing']['Num Integrations Out of Transit'], data['timing']['Num Integrations In Transit']

In [4]:
variance_delta = (((N_out / N_in) * (1. / S_out))**2) * var_in + \
                 (((N_out / N_in) * (S_in / S_out**2))**2) * var_out

variance_Fj = (var_out * N_out)/(S_out**2)

In [5]:
def transit_model(t, inc, w):
    """
    Given a time array t, an inclination "inc" and a wavelength in microns "w", this function returns 
    a lightcurve model including limb-darkening.
    """
    params = batman.TransitParams()
    params.t0 = 0. # time of inferior conjunction
    params.per = P # orbital period (days)
    params.a = a # semi-major axis (in units of stellar radii)
    params.rp = rp
    params.inc = inc # orbital inclination (in degrees)
    params.ecc = ecc # eccentricity
    params.w = omega # longitude of periastron (in degrees) p
    params.limb_dark = 'quadratic' # limb darkening profile to use
    if w<np.min(wld):
        c1,c2 = f1(np.min(wld)),f2(np.min(wld))
    elif w>np.max(wld):
        c1,c2 = f1(np.max(wld)),f2(np.max(wld))
    else:
        c1,c2 = f1(w), f2(w)
    
    params.u = [c1,c2] # limb darkening coefficients

    tmodel = batman.TransitModel(params, t.astype('float64'))
    return tmodel.light_curve(params)

In [6]:
# First, create an array that goes from zero to 927. Multiply that by the time per integration (in days):
time_per_int = data['timing']['Time/Integration incl reset (sec)']/(3600.*24.)
nintegrations = data['timing']['APT: Num Integrations per Occultation']
t = np.arange(nintegrations) * time_per_int

# Now, if nintegrations is even, substract the mean between elements (nintegrations/2) and  (nintegrations/2)+1. 
# If odd, substract np.floor(nintegrations/2):
if nintegrations % 2 == 0:
    middle_mean = (t[int(nintegrations/2)] + t[int(nintegrations/2) + 1])*0.5
else:
    middle_mean = t[int(np.floor(nintegrations/2))]
t = t - middle_mean

In [7]:
P, a, inc, ecc, omega, rp = 4.4652998, 10.22, 85.79, 0., 90., 0.116

wld,u1,u2 = np.loadtxt('data/lds_order1.txt',unpack=True)
f1 = interp1d(wld,u1)
f2 = interp1d(wld,u2)

w_PandExo, deptherror_PandExo = data['FinalSpectrum']['wave'], data['FinalSpectrum']['error_w_floor']

In [8]:
# Generate models for each wavelength bin.
model_per_wavelength = [transit_model(t, inc, wavelength) for wavelength in w_PandExo]

In [9]:
lightcurve_errors = np.sqrt(variance_Fj)

# Generate array of noise for each wavelength bin.
noise_per_model = [np.random.normal(0., lightcurve_errors[i], len(t)) for i in range(len(lightcurve_errors))]

In [10]:
# Name of the parameters to be fit:
params = ['P_p1','t0_p1','p_p1','b_p1','q1_SOSS','q2_SOSS','ecc_p1','omega_p1',\
              'a_p1', 'mdilution_SOSS', 'mflux_SOSS', 'sigma_w_SOSS']

# Distributions:
dists = ['fixed','fixed','uniform','fixed','uniform','uniform','fixed','fixed',\
                 'fixed', 'fixed', 'normal', 'fixed']

# Hyperparameters
hyperps = [P, 0., [0.,1], a*np.cos(inc*np.pi/180.), [0., 1.], [0., 1.], ecc, omega,\
                   a, 1.0, [0.,0.1], 1e-6]

priors = juliet.generate_priors(params,dists,hyperps)

In [ ]:
# Create dictionaries for the times; note for the errors on the fluxes, we assume they are all the same:
times, fluxes, fluxes_error = {}, {}, {}
dataset_per_wavelength = []

for wavelength, model, noise, error in zip(w_PandExo, model_per_wavelength, noise_per_model, lightcurve_errors):
    print(wavelength)
    times[wavelength], fluxes[wavelength], fluxes_error[wavelength] = t, model + noise, \
                                                          np.ones(len(t))*error
    # Load the dataset 
    dataset = juliet.load(priors=priors, t_lc = times, y_lc = fluxes,\
                            yerr_lc = fluxes_error, out_folder = 'lowest_wavelength_results')
    dataset_per_wavelength.append(dataset)

0.8348
0.8357
0.8367
0.8377
0.8386
0.8396
0.8406
0.8415
0.8425
0.8435
0.8445
0.8454
0.8464
0.8474
0.8483
0.8493
0.8503
0.8512
0.8522
0.8532
0.8542
0.8551
0.8561
0.8571
0.858
0.859
0.86
0.861
0.8619
0.8629
0.8639
0.8648
0.8658
0.8668
0.8677
0.8687
0.8697
0.8707
0.8716
0.8726
0.8736
0.8745
0.8755
0.8765
0.8774
0.8784
0.8794
0.8803
0.8813
0.8823
0.8833
0.8842
0.8852
0.8862
0.8871
0.8881
0.8891
0.89
0.891
0.892
0.893
0.8939
0.8949
0.8959
0.8968
0.8978
0.8988
0.8998
0.9007
0.9017
0.9027
0.9036
0.9046
0.9056
0.9065
0.9075
0.9085
0.9095
0.9104
0.9114
0.9124
0.9133
0.9143
0.9153
0.9162
0.9172
0.9182
0.9192
0.9201
0.9211
0.9221
0.923
0.924
0.925
0.9259
0.9269
0.9279
0.9288
0.9298
0.9308
0.9318
0.9327
0.9337
0.9347
0.9356
0.9366
0.9376
0.9385
0.9395
0.9405
0.9415
0.9424
0.9434
0.9444
0.9453
0.9463
0.9473
0.9482
0.9492
0.9502
0.9512
0.9521
0.9531
0.9541
0.955
0.956
0.957
0.958
0.9589
0.9599
0.9609
0.9618
0.9628
0.9638
0.9647
0.9657
0.9667
0.9677
0.9686
0.9696
0.9706
0.9715
0.9725
0.9735
0.9744
0.

In [ ]:
results_lowest = dataset.fit()

In [ ]:
modelfit_lowest = results_lowest.lc.evaluate('dsfjkh')

In [ ]:
plt.figure(figsize=(15,5))

# Plot simulated data, along with the real model and the fitted model:
plt.plot(t, model_lowest + noise_lowest, '.', color='cornflowerblue')
plt.plot(t, model_lowest, color='cornflowerblue', lw = 3,label='Real model')
plt.plot(t, modelfit_lowest, color='black', lw = 3,label='Fitted model')
plt.ylim(0.980,1.005)
plt.xlim(-0.13,0.13)

In [ ]:
p = results_lowest.posteriors['posterior_samples']['p_p1']

In [ ]:
# Estimate from our fit:
depth = p**2
error_on_depth = np.sqrt(np.var(depth))
juliet_error = np.round(error_on_depth*1e6,1)
# Estimate from PandExo:
pandexo_error = np.round(deptherror_PandExo[0]*1e6,1)

In [ ]:
(juliet_error - pandexo_error) / juliet_error